# Chapter 9 - Deploying and using models in production

This Jupyter notebook provides a resource to help you follow the code examples from the book more easily. The notebook covers all practical code snippets and exercises found in: Chapter 9 - Deploying and using models in production

## Self-serving Flair models

### Installing required libraries

In [ ]:
!pip install flask==1.1.1

### Defining the sequence labelling function

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

tagger = SequenceTagger.load('ner')

def tag_text(input_text):
    sentence = Sentence(input_text)
    tagger.predict(sentence)
    
    return sentence.to_tagged_string()

### Defining the endpoint

In [ ]:
from flask import Flask, jsonify, request

app = Flask(__name__)

@app.route('/ner-service')
def query_example():
    text = request.args.get('input', '')
    
    tagged_text = tag_text(text)

    return jsonify({"result": tagged_text})

### Starting the service

In [ ]:
app.run(debug=False, port=5002)

# visit http://localhost:5002/ner-service?input=Never+been+to+Lima to see resuls

## Using managed services for deploying and using Flair models

### Hugging Face Models Hub

#### Training a mock Flair model

In [ ]:
from flair.datasets import UD_ENGLISH

corpus = UD_ENGLISH().downsample(0.001)

lbl_type = 'upos'
label_dict = corpus.make_label_dictionary(label_type=lbl_type)

In [ ]:
from flair.embeddings import CharacterEmbeddings

# lightweight embeddings to get a tiny, but useless model
embedding = CharacterEmbeddings()

In [ ]:
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

# one-dimensional hidden_size to produce a tiny model
tagger = SequenceTagger(embeddings=embedding,
                        tag_dictionary=label_dict,
                        tag_type=lbl_type,
                        hidden_size=1)

trainer = ModelTrainer(tagger, corpus)

# train for only one epoch as quality does not matter here 
trainer.train('tiny_model', max_epochs=1)

#### Using the model as part of the Accelerated Inference API

In [ ]:
import requests

API_URL = "https://api-inference.huggingface.co/models/flairbook/flairmodel"  # make sure to replace the url
h = {"Authorization": "Bearer <TOKEN>"}  # make sure to replace the token

def query(payload):
    response = requests.post(API_URL, headers=h, json=payload)
    return response.json()

print(query({"inputs": "abc def"}))


#### Using public Hugging Face hub models inside Flair

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

tagger = SequenceTagger.load('flairbook/flairmodel')  # make sure to use your own model key

sentence = Sentence('abc')
tagger.predict(sentence)

print(sentence.to_tagged_string())